Как ставить написано в документации и документации.

Предварительно проверьте, что у вас установлена Java 11.

neo4j-admin set-initial-password ANewPassword

В файле Neo4J.conf необходимо раскомментировать следующий ключ. dbms.security.auth_enabled=false Перезапускаем СУБД.

Теперь заходим в админку базы данных. http://127.0.0.1:7474/

In [1]:
from neo4j import GraphDatabase
from flask import Flask
from flask import url_for, render_template, request, redirect

import re

In [5]:
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"), encrypted=False)

In [117]:
app = Flask(__name__)

In [118]:
session = {}
session['username'] = None

In [119]:
@app.route('/home', methods=['GET', 'POST'])
def index():
    if session['username'] == None:
        return render_template('home.html')
    else:
        tasks = []
        importance = []
        statuses = []
        ids = []
        
        if request.method == "POST":
            det = request.form
            importance_list = []
            status_list = []
            if 'importance1' in det:
                importance_list.append(det['importance1'])
            if 'importance2' in det:
                importance_list.append(det['importance2'])
            if 'importance3' in det:
                importance_list.append(det['importance3'])
    
            if 'status1' in det:
                status_list.append(det['status1'])
            if 'status2' in det:
                status_list.append(det['status2'])
            if 'status3' in det:
                status_list.append(det['status3'])
           
            if not importance_list and not status_list:
                c = driver.session().run("MATCH (f {nickname: $nickname})-[:HAS_TASK]->(t) return t.task_name, t.importance, t.status, t.deadline", nickname=session['username'])
            else:
                base_string = 'MATCH (f {nickname: $nickname})-[:HAS_TASK]->(t)'
                if importance_list or status_list:
                    base_string += ' WHERE ('
                if importance_list:
                    for i, element in enumerate(importance_list):
                        base_string = base_string + 't.importance = \'' + element + '\''
                        if i != len(importance_list) - 1:
                            base_string += ' OR '
                        else:
                            base_string += ')'
                if status_list:
                    if importance_list:
                        base_string += ' AND ('
                    for i, element in enumerate(status_list):
                        base_string = base_string + 't.status = \'' + element + '\''
                        if i != len(status_list) - 1:
                            base_string += ' OR '
                        else:
                            base_string += ')'
            base_string += ' RETURN t.task_name, t.importance, t.status, t.deadline'
            c = driver.session().run(base_string, nickname = session['username'])
        else:
            c = driver.session().run("MATCH (f {nickname: $nickname})-[:HAS_TASK]->(t) return t.task_name, t.importance, t.status, t.deadline", nickname=session['username'])
        data = []
        for i in c.data():
                elem = [i['t.task_name'], i['t.importance'], i['t.status'], i['t.deadline']]
                data.append(elem)
        return render_template('home_logged_in.html', data = data, username = session['username'])

In [120]:
@app.route('/signup', methods=['GET', 'POST'])
def sign_up():
    if request.method == "POST":
        username = request.form['username']
        password = request.form['password']
        c = driver.session().run("match (n {nickname: $nickname}) return n.nickname", nickname = username)
        if c.data():
            return render_template('user_exists.html')
        else:
            driver.session().run("CREATE (f:User {nickname: $nickname, password: $password, n_of_friends: 0, n_of_requests: 0})", nickname = username, password = password)
            return redirect('login')
    return render_template('sign_up.html')

In [121]:
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == "POST":
        username = request.form['username']
        password = request.form['password']
        c = driver.session().run("match (n {nickname: $nickname}) return n.nickname, n.password", nickname = username)
        result = c.data()
        if result:
            if result[0]['n.password'] == password:
                session['username'] = username
                return redirect('home')
            else:
                return render_template('wrong_password.html')
        else:
            return render_template('no_user.html')
    return render_template('login.html')

In [122]:
@app.route('/new_task', methods=['GET', 'POST'])
def new_task(): 
    if request.method == "POST":
        details = request.form
        
        c = driver.session().run('MATCH (f {nickname: $nickname})\
                                CREATE (t:Task {task_name: $task_name, importance: $importance, status: $status, deadline: $deadline})\
                                CREATE (f)-[:HAS_TASK]->(t)', nickname = session['username'], task_name = details['tasks'], importance = details['importance'], status = details['status'], deadline = '14.03.2021')
        return redirect('home')
    importance = ['не очень важно', 'важно', 'очень важно']
    status = ['не начато', 'в процессе', 'закончено']
    return render_template('new_task.html', status = status, importance = importance)

In [123]:
@app.route('/delete', methods=['POST', 'GET'])
def delete():
    data = request.args.get('data').strip('\[\]')
    as_list = data.split(', ')
    d = [i.strip('\'') for i in as_list]
    c = driver.session().run('MATCH (f {nickname: $nickname})-[:HAS_TASK]->(t) WHERE t.task_name = $task_name AND t.importance = $importance AND t.status = $status\
                            DETACH DELETE t',
                            nickname = session['username'], task_name = d[0], importance = d[1], status = d[2])
    return redirect('home')

In [124]:
@app.route('/update', methods=['GET', 'POST'])
def update_task():
    data = request.args.get('data').strip('\[\]')
    as_list = data.split(', ')
    d = [i.strip('\'') for i in as_list]
    if request.method == "POST":
        details = request.form
        
        driver.session().run('MATCH (f {nickname: $nickname})-[:HAS_TASK]->(t) WHERE t.task_name = $task_name AND t.importance = $importance AND t.status = $status\
                            SET t.importance = $new_importance, t.status = $new_status',
                            nickname = session['username'], task_name = d[0], importance = d[1], status = d[2],
                            new_importance = details['importance'], new_status = details['status'])
        return redirect('home')
    
    importance = ['не очень важно', 'важно', 'очень важно']
    status = ['не начато', 'в процессе', 'закончено']
    return render_template('update_task.html', status = status, importance = importance, task = d[0])

In [125]:
@app.route('/friends', methods = ['POST', 'GET'])
def friends():
    c = driver.session().run("MATCH (f {nickname: $nickname})-[:FRIEND_OF]->(m) return m.nickname", nickname = session['username'])
    data = c.data()
    if data:
        friends_usernames = [result['m.nickname'] for result in data]
    else:
        friends_usernames = []
        
    c = driver.session().run("MATCH (f {nickname: $nickname})<-[:SENT_REQUEST_TO]-(m) return m.nickname", nickname = session['username'])
    data = c.data()
    if data:
        requests = [result['m.nickname'] for result in data]
    else:
        requests = []
    return render_template('friends.html', usernames = friends_usernames, requests = requests)

In [126]:
@app.route('/send_request', methods = ["POST", "GET"])
def send_request():
    if request.args.get('username'):
        username = request.args.get('username')
        if username == session['username']:
            return redirect('/home')
        res = driver.session().run("MATCH (f {nickname: $nickname}) RETURN f.nickname", nickname = username)
        if res:
            driver.session().run("MATCH (f {nickname: $nickname1})\
                                MATCH (m {nickname:$nickname2})\
                                CREATE (f)-[:SENT_REQUEST_TO]->(m)",
                                nickname1 = session['username'], nickname2 = username)
            return redirect('/friends')
        else:
            return render_template('no_user.html')
    else:    
    
        return ('Не получил юзернейм.')

In [127]:
@app.route('/reply_to_request', methods = ["POST", "GET"])
def reply_to_request():
    if request.args.get('friend_req'):
        username = str(request.args.get('nickname'))
        with open ('m.txt', 'w') as f:
            f.write(username)
        if request.args.get('friend_req') == 'Принять':
            driver.session().run("MATCH (f {nickname: $nickname1})\
                                MATCH (m {nickname:$nickname2})\
                                CREATE (f)-[:FRIEND_OF]->(m)\
                                CREATE (f)<-[:FRIEND_OF]-(m)",
                                nickname1 = session['username'], nickname2 = username)
        driver.session().run("MATCH (f {nickname: $nickname1})<-[r:SENT_REQUEST_TO]-(m {nickname: $nickname2})\
                            DELETE r",
                            nickname1 = session['username'], nickname2 = username)
        return redirect('/friends')
    else:
        return ('You should not be here')

In [128]:
@app.route('/<username>', methods = ["POST", "GET"])
def profile(username):
    c = driver.session().run("MATCH (n {nickname: $nickname}) return n.nickname", nickname = username)
    if c.data():
        t = driver.session().run("MATCH (f {nickname: $nickname})-[:HAS_TASK]->(t) return t.task_name, t.importance, t.status, t.deadline", nickname=username)
        data = []
        for i in t.data():
            elem = [i['t.task_name'], i['t.importance'], i['t.status'], i['t.deadline']]
            data.append(elem)
        
        if session['username']:
            t = driver.session().run("MATCH (f {nickname: $nickname1})-[:FRIEND_OF]->(m {nickname: $nickname2}) RETURN f", nickname1 = session['username'], nickname2 = username)
            if t.data():
                friend_str = 'Вы дружите с этим пользователем.'
            else:
                t = driver.session().run("Match path = shortestpath( (f:User {nickname:$nickname1}) –[r:FRIEND_OF *]->(p:User {nickname:$nickname2})) return length(path) as len", nickname1 = session['username'], nickname2 = username)
                data = t.data()
                if not data:
                    num = 0
                else:
                    num = data[0]['len']
                if (num % 100 >= 11 and num % 100 <= 19) or (num % 10 >= 5 and num % 10 <= 9) or (num % 10 == 0):
                    handshakes = ' рукопожатий.'
                elif num % 10 == 1:
                    handshakes = ' рукопожатие.'
                else:
                    handshakes = ' рукопожатия.'
                friend_str = 'Вы связаны с этим пользователем через ' + str(num) + handshakes
        else:
            friend_str = 'Войдите в систему, чтобы узнать, через сколько рукопожатий вы связаны с этим пользователем.'
        return render_template('user.html', username = username, data = data, friend_str = friend_str)
    else:
        return render_template('no_user.html')

In [129]:
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Mar/2020 14:13:43] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2020 14:13:44] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [31/Mar/2020 14:13:44] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2020 14:13:49] "GET /mcqueen HTTP/1.1" 200 -


In [100]:
c = driver.session().run("Match path = shortestpath( (f:User {nickname:$nickname1}) –[r:FRIEND_OF *]->(p:User {nickname:$nickname2})) return length(path) as len", nickname1 = 'fishowl', nickname2 = 'kolbasa')

In [101]:
d = c.data()

In [103]:
print(d)

[]


In [87]:
len(d[0])

1